# Predicting Boston Housing Prices

## Updating a model using SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

In this notebook, we will continue working with the [Boston Housing Dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). Our goal in this notebook will be to train two different models and to use SageMaker to switch a deployed endpoint from using one model to the other. One of the benefits of using SageMaker to do this is that we can make the change without interrupting service. What this means is that we can continue sending data to the endpoint and at no point will that endpoint disappear.

## General Outline

The typical steps are:
1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

In this notebook we will be skipping step 5, testing the model. In addition, we will perform steps 4, 6 and 7 multiple times with different models. To start, we will look at performing an A/B test between two different models. Then, once we've decided on a model to use, updating the existing endpoint so that it only sends data to a single model.

This time we are using a **hybrid approach, including both the high level and low level functionality**. In this case we use the high level approach to train a model (to produce model artifacts) and then we use the low level approach to construct the model itself and to construct the endpoint configuration. The reason for this is so that we can have more control over how our endpoint behaves.

## Step 0: Setting up the notebook

We begin by setting up all of the necessary bits required to run our notebook. To start that means loading all of the Python modules we will need.

In [3]:
import os
import numpy as np
import pandas as pd

from pprint import pprint
from time import gmtime, strftime

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.datasets import load_boston
import sklearn.model_selection

In [4]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
session = sagemaker.Session()

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = get_execution_role()

## Step 1: Downloading the data

In [5]:
boston = load_boston()

## Step 2: Preparing and splitting the data

In [6]:
# First we package up the input data and the target variable (the median value) as pandas dataframes. This
# will make saving the data to a file a little easier later on.

X_bos_pd = pd.DataFrame(boston.data, columns=boston.feature_names)
Y_bos_pd = pd.DataFrame(boston.target)

# We split the dataset into 2/3 training and 1/3 testing sets.
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_bos_pd, Y_bos_pd, test_size=0.33)

# Then we split the training set further into 2/3 training and 1/3 validation sets.
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

## Step 3: Uploading the training and validation files to S3

When a training job is constructed using SageMaker, a container is executed which performs the training operation. This container is given access to data that is stored in S3. This means that we need to upload the data we want to use for training to S3. We can use the SageMaker API to do this and hide some of the details.

### Save the data locally

First we need to create the train and validation csv files which we will then upload to S3.

In [7]:
# This is our local data directory. We need to make sure that it exists.
data_dir = '../data/boston'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [8]:
# We use pandas to save our train and validation data to csv files. Note that we make sure not to include header
# information or an index as this is required by the built in algorithms provided by Amazon. Also, it is assumed
# that the first entry in each row is the target variable.

pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Upload to S3

In [9]:
prefix = 'boston-update-endpoints'

val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## Step 4 (A): Train the XGBoost model

In [12]:
# As stated above, we use this utility method to construct the image name for the training container.
xgb_container = get_image_uri(session.boto_region_name, 'xgboost', '0.90-1')

# Now that we know which container to use, we can construct the estimator object.
xgb = sagemaker.estimator.Estimator(xgb_container, # The name of the training container
                                    role,      # The IAM role to use (our current role in this case)
                                    train_instance_count=1, # The number of instances to use for training
                                    train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                        # Where to save the output (the model artifacts)
                                    sagemaker_session=session) # The current SageMaker session

In [13]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)

In [14]:
# This is a wrapper around the location of our train and validation data, to make sure that SageMaker
# knows our data is in csv format.
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='text/csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='text/csv')

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2019-12-27 05:42:37 Starting - Starting the training job...
2019-12-27 05:42:39 Starting - Launching requested ML instances......
2019-12-27 05:43:40 Starting - Preparing the instances for training......
2019-12-27 05:44:57 Downloading - Downloading input data
2019-12-27 05:44:57 Training - Downloading the training image...
2019-12-27 05:45:37 Uploading - Uploading generated training model
2019-12-27 05:45:37 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[05:45:27] 227x13 matrix with 2951 ent

## Step 5: Test the trained model

We will be skipping this step for now.


## Step 6 (A): Deploy the trained model

Even though we used the high level approach to construct and train the XGBoost model, we will be using the lower level approach to deploy it. One of the reasons for this is so that we have additional control over how the endpoint is constructed.

### Build the model object

Of course, before we can deploy the model, we need to first create it. The `fit` method that we used earlier created some _model artifacts_ and we can use these to construct a **model object**.

In [15]:
# Remember that a model needs to have a unique name
xgb_model_name = "boston-update-xgboost-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# We also need to tell SageMaker which container should be used for inference and where it should
# retrieve the model artifacts from. In our case, the xgboost container that we used for training
# can also be used for inference and the model artifacts come from the previous call to fit.
xgb_primary_container = {
    "Image": xgb_container,
    "ModelDataUrl": xgb.model_data
}

# And lastly we construct the SageMaker model
xgb_model_info = session.sagemaker_client.create_model(
                                ModelName = xgb_model_name,
                                ExecutionRoleArn = role,
                                PrimaryContainer = xgb_primary_container)

### Create the endpoint configuration

Once we have a model we can start putting together the endpoint. Recall that to do this we need to first create an endpoint configuration, essentially the blueprint that SageMaker will use to build the endpoint itself.

In [16]:
# As before, we need to give our endpoint configuration a name which should be unique
xgb_endpoint_config_name = "boston-update-xgboost-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
xgb_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = xgb_endpoint_config_name,
                            ProductionVariants = [{
                                "InstanceType": "ml.m4.xlarge",
                                "InitialVariantWeight": 1,
                                "InitialInstanceCount": 1,
                                "ModelName": xgb_model_name,
                                "VariantName": "XGB-Model"
                            }])

### Deploy the endpoint

Now that the endpoint configuration has been created, we can ask SageMaker to build our endpoint.

In [17]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = xgb_endpoint_config_name)

In [18]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

--------------------------------------------------------------------------------------------------------------!

## Step 7 (A): Use the model

Now that our model is trained and deployed we can send some test data to it and evaluate the results.

In [19]:
# Send first row only
response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[0])))

In [20]:
pprint(response)

{'Body': <botocore.response.StreamingBody object at 0x7fd25289aba8>,
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'XGB-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '17',
                                      'content-type': 'text/csv; charset=utf-8',
                                      'date': 'Fri, 27 Dec 2019 05:55:30 GMT',
                                      'x-amzn-invoked-production-variant': 'XGB-Model',
                                      'x-amzn-requestid': 'df14296c-6f83-45f6-ba56-ac6a2e389824'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'df14296c-6f83-45f6-ba56-ac6a2e389824',
                      'RetryAttempts': 0}}


In [21]:
result = response['Body'].read().decode("utf-8")

In [22]:
pprint(result)

'18.72989845275879'


In [23]:
Y_test.values[0]

array([17.6])

## Shut down the endpoint

Now that we know that the XGBoost endpoint works, we can shut it down. We will make use of it again later.

In [ ]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

## Step 4 (B): Train the Linear model

Suppose we are working in an environment where the XGBoost model that we trained earlier is becoming too costly. Perhaps the number of calls to our endpoint has increased and the length of time it takes to perform inference with the XGBoost model is becoming problematic. A possible solution might be to train a simpler model to see if it performs nearly as well. In our case, we will construct a linear model. The process of doing this is the same as for creating the XGBoost model that we created earlier, although there are different hyperparameters that we need to set.

In [24]:
# Similar to the XGBoost model, we will use the utility method to construct the image name for the training container.
linear_container = get_image_uri(session.boto_region_name, 'linear-learner')

# Now that we know which container to use, we can construct the estimator object.
linear = sagemaker.estimator.Estimator(linear_container, # The name of the training container
                                        role,      # The IAM role to use (our current role in this case)
                                        train_instance_count=1, # The number of instances to use for training
                                        train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
                                        output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                            # Where to save the output (the model artifacts)
                                        sagemaker_session=session) # The current SageMaker session

Before asking SageMaker to train our model, we need to set some hyperparameters. In this case we will be using a linear model so the number of hyperparameters we need to set is much fewer. For more details see the [Linear model hyperparameter page](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)

In [25]:
linear.set_hyperparameters(feature_dim=13, # Our data has 13 feature columns
                           predictor_type='regressor', # We wish to create a regression model
                           mini_batch_size=200) # Here we set how many samples to look at in each iteration

In [26]:
linear.fit({'train': s3_input_train, 'validation': s3_input_validation})

2019-12-27 05:55:30 Starting - Starting the training job...
2019-12-27 05:55:31 Starting - Launching requested ML instances...
2019-12-27 05:56:28 Starting - Preparing the instances for training......
2019-12-27 05:57:22 Downloading - Downloading input data...
2019-12-27 05:57:55 Training - Downloading the training image...
2019-12-27 05:58:27 Uploading - Uploading generated training model
2019-12-27 05:58:27 Completed - Training job completed
Docker entrypoint called with argument(s): train
[12/27/2019 05:58:17 INFO 140369721902912] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': 

Training seconds: 65
Billable seconds: 65


## Step 6 (B): Deploy the trained model

### Build the model

In [27]:
# First, we create a unique model name
linear_model_name = "boston-update-linear-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# We also need to tell SageMaker which container should be used for inference and where it should
# retrieve the model artifacts from. In our case, the linear-learner container that we used for training
# can also be used for inference.
linear_primary_container = {
    "Image": linear_container,
    "ModelDataUrl": linear.model_data
}

# And lastly we construct the SageMaker model
linear_model_info = session.sagemaker_client.create_model(
                                ModelName = linear_model_name,
                                ExecutionRoleArn = role,
                                PrimaryContainer = linear_primary_container)

### Create the endpoint configuration

In [28]:
# As before, we need to give our endpoint configuration a name which should be unique
linear_endpoint_config_name = "boston-linear-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
linear_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = linear_endpoint_config_name,
                            ProductionVariants = [{
                                "InstanceType": "ml.m4.xlarge",
                                "InitialVariantWeight": 1,
                                "InitialInstanceCount": 1,
                                "ModelName": linear_model_name,
                                "VariantName": "Linear-Model"
                            }])

### Deploy the endpoint

In [29]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = linear_endpoint_config_name)

In [33]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

---------------------------------------------------------------------!

## Step 7 (B): Use the model

Just like with the XGBoost model, we will send some data to our endpoint to make sure that it is working properly.

**Note:** It is important to notice that the result returned by the linear learner model is json, compared to the csv data returned by the XGBoost model. You can't always assume that different models will return data in the same way although typically the return type is specified in the documentation.

In [34]:
response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[0])))

In [35]:
pprint(response)

{'Body': <botocore.response.StreamingBody object at 0x7fd2528a6d30>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'Linear-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '48',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 27 Dec 2019 06:06:18 GMT',
                                      'x-amzn-invoked-production-variant': 'Linear-Model',
                                      'x-amzn-requestid': 'ce4ff9c7-b4e4-4bdb-8789-72a4d2d91861'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'ce4ff9c7-b4e4-4bdb-8789-72a4d2d91861',
                      'RetryAttempts': 0}}


In [36]:
result = response['Body'].read().decode("utf-8")

In [37]:
pprint(result)

'{"predictions": [{"score": 11.961992263793945}]}'


In [38]:
Y_test.values[0]

array([17.6])

## Shut down the endpoint

Now that we know that the Linear model's endpoint works, we can shut it down.

In [39]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': '7c2657d8-8faa-4d75-98c6-1ef744cee3b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7c2657d8-8faa-4d75-98c6-1ef744cee3b0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 27 Dec 2019 06:06:17 GMT'},
  'RetryAttempts': 0}}

## Step 6 (C): Deploy a combined model (A/B-Testing)

So far we've constructed two separate models which we could deploy and use. Before we talk about how we can change a deployed endpoint from one configuration to another, let's consider a slightly different situation. Suppose that before we switch from using only the XGBoost model to only the Linear model, we first want to do something like an A-B test, where we send some of the incoming data to the XGBoost model and some of the data to the Linear model.

To actually get SageMaker to do this for us is not too different from deploying a model in the way that we've already done. The only difference is that we need to list more than one model in the production variants parameter of the endpoint configuration.

A reasonable question to ask is, how much data is sent to each of the models that I list in the production variants parameter? The answer is that it depends on the weight set for each model.

Suppose that we have $k$ models listed in the production variants and that each model $i$ is assigned the weight $w_i$. Then each model $i$ will receive $w_i / W$ of the traffic where $W = \sum_{i} w_i$.

In our case, since we have two models, the linear model and the XGBoost model, and each model has weight 1, we see that each model will get 1 / (1 + 1) = 1/2 of the data sent to the endpoint.

In [40]:
# As before, we need to give our endpoint configuration a name which should be unique
combined_endpoint_config_name = "boston-combined-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
combined_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = combined_endpoint_config_name,
                            ProductionVariants = [
                                { # First we include the linear model
                                    "InstanceType": "ml.m4.xlarge",
                                    "InitialVariantWeight": 1,
                                    "InitialInstanceCount": 1,
                                    "ModelName": linear_model_name,
                                    "VariantName": "Linear-Model"
                                }, { # And next we include the xgb model
                                    "InstanceType": "ml.m4.xlarge",
                                    "InitialVariantWeight": 1,
                                    "InitialInstanceCount": 1,
                                    "ModelName": xgb_model_name,
                                    "VariantName": "XGB-Model"
                                }])

In [41]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = combined_endpoint_config_name)

In [42]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

--------------------------------------------------------------------------------------------------------------!

## Step 7 (C): Use the model

Now that we've constructed an endpoint which sends data to both the XGBoost model and the linear model we can send some data to the endpoint and see what sort of results we get back.

In [43]:
for rec in range(10):
    response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[rec])))
    pprint(response)
    result = response['Body'].read().decode("utf-8")
    print(result)
    print(Y_test.values[rec])

{'Body': <botocore.response.StreamingBody object at 0x7fd2528412b0>,
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'XGB-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '17',
                                      'content-type': 'text/csv; charset=utf-8',
                                      'date': 'Fri, 27 Dec 2019 06:19:55 GMT',
                                      'x-amzn-invoked-production-variant': 'XGB-Model',
                                      'x-amzn-requestid': '75fb986d-51b3-4762-9e41-cc5dfa8bbbb2'},
                      'HTTPStatusCode': 200,
                      'RequestId': '75fb986d-51b3-4762-9e41-cc5dfa8bbbb2',
                      'RetryAttempts': 0}}
18.72989845275879
[17.6]
{'Body': <botocore.response.StreamingBody object at 0x7fd2528412e8>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'Linear-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '48',
                                     

If at some point we aren't sure about the properties of a deployed endpoint, we can use the `describe_endpoint` function to get SageMaker to return a description of the deployed endpoint.

In [44]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2019, 12, 27, 6, 10, 40, 354000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:eu-west-1:873674308518:endpoint/boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointConfigName': 'boston-combined-endpoint-config-2019-12-27-06-10-39',
 'EndpointName': 'boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2019, 12, 27, 6, 19, 51, 250000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2019, 12, 27, 6, 10, 42, 820000, tzinfo=tzlocal()),
                                             'ResolvedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner@sha256:0968670c29a5901e4cf38d3e690c3550e6c38539e0f004ccc79bb1e56d8b452b',
                                             'SpecifiedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com

## Updating an Endpoint

Now suppose that we've done our A-B test and the new linear model is working well enough. What we'd like to do now is to switch our endpoint from sending data to both the XGBoost model and the linear model to sending data only to the linear model.

Of course, we don't really want to shut down the endpoint to do this as doing so would interrupt service to whoever depends on our endpoint. Instead, we can ask SageMaker to **update** an endpoint to a new endpoint configuration.

What is actually happening is that SageMaker will set up a new endpoint with the new characteristics. Once this new endpoint is running, SageMaker will switch the old endpoint so that it now points at the newly deployed model, making sure that this happens seamlessly in the background.

In [45]:
session.sagemaker_client.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=linear_endpoint_config_name)

{'EndpointArn': 'arn:aws:sagemaker:eu-west-1:873674308518:endpoint/boston-update-endpoint-2019-12-27-06-10-40',
 'ResponseMetadata': {'RequestId': '124f6f14-0853-421a-8a5e-cc84e99d5b7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '124f6f14-0853-421a-8a5e-cc84e99d5b7a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Fri, 27 Dec 2019 06:19:55 GMT'},
  'RetryAttempts': 0}}

To get a glimpse at what is going on, we can ask SageMaker to describe our in-use endpoint now, before the update process has completed. When we do so, we can see that the in-use endpoint still has the same characteristics it had before.

In [46]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2019, 12, 27, 6, 10, 40, 354000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:eu-west-1:873674308518:endpoint/boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointConfigName': 'boston-combined-endpoint-config-2019-12-27-06-10-39',
 'EndpointName': 'boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointStatus': 'Updating',
 'LastModifiedTime': datetime.datetime(2019, 12, 27, 6, 19, 55, 807000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2019, 12, 27, 6, 10, 42, 820000, tzinfo=tzlocal()),
                                             'ResolvedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner@sha256:0968670c29a5901e4cf38d3e690c3550e6c38539e0f004ccc79bb1e56d8b452b',
                                             'SpecifiedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/

If we now wait for the update process to complete, and then ask SageMaker to describe the endpoint, it will return the characteristics of the new endpoint configuration.

In [47]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

--------------------------------------------------------------------------------------!

In [48]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2019, 12, 27, 6, 10, 40, 354000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:eu-west-1:873674308518:endpoint/boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointConfigName': 'boston-linear-endpoint-config-2019-12-27-05-58-42',
 'EndpointName': 'boston-update-endpoint-2019-12-27-06-10-40',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2019, 12, 27, 6, 27, 8, 628000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2019, 12, 27, 6, 19, 58, 176000, tzinfo=tzlocal()),
                                             'ResolvedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner@sha256:0968670c29a5901e4cf38d3e690c3550e6c38539e0f004ccc79bb1e56d8b452b',
                                             'SpecifiedImage': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/li

## Shut down the endpoint

Now that we've finished, we need to make sure to shut down the endpoint.

In [49]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': '976123bd-dd17-4e63-a88f-a287205d4af5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '976123bd-dd17-4e63-a88f-a287205d4af5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 27 Dec 2019 06:27:11 GMT'},
  'RetryAttempts': 0}}

## Optional: Clean up

In [50]:
!rm $data_dir/*
!rmdir $data_dir

---